In [3]:
# --- Imports ---
import os
import time
import numpy as np
import cv2
from matplotlib import pyplot as plt
import mediapipe as mp

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau

# --- Configuration ---
DATA_PATH = 'MP_Data'  # base path
actions = np.array(['hello', 'thanks', 'iloveyou'])
sequence_length = 30
no_sequences = 30
log_dir = 'Logs'
os.makedirs(log_dir, exist_ok=True)

# --- Label mapping ---
label_map = {label: idx for idx, label in enumerate(actions)}

# --- TensorBoard + Early stopping callbacks ---
tb_callback = TensorBoard(log_dir=log_dir)
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)

# --- Load data ---
sequences, labels = [], []

for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    if not os.path.exists(action_path):
        print(f"⚠️ Skipping missing folder: {action_path}")
        continue

    sequence_dirs = [d for d in os.listdir(action_path) if os.path.isdir(os.path.join(action_path, d))]
    if not sequence_dirs:
        print(f"⚠️ No sequences found for {action}")
        continue

    for sequence in sorted(sequence_dirs):
        seq_path = os.path.join(action_path, sequence)
        frames = []
        for frame_num in range(sequence_length):
            file_path = os.path.join(seq_path, f"{frame_num}.npy")
            if not os.path.exists(file_path):
                print(f"⚠️ Missing frame {frame_num} in {seq_path}, skipping this sequence.")
                frames = []  # discard incomplete sequence
                break
            frames.append(np.load(file_path))
        if frames:  # only add complete sequences
            sequences.append(frames)
            labels.append(label_map[action])

if not sequences or not labels:
    raise RuntimeError("❌ No valid training data found in MP_Data/. Check that folders and .npy files exist.")

X = np.array(sequences)
y = to_categorical(labels, num_classes=len(actions)).astype(int)
print(f"✅ Loaded {len(X)} sequences successfully.")


# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, shuffle=True, random_state=42)

# Model Definition (Using Default Tanh for LSTM) ---
model = Sequential([
    # Remove 'activation=relu' to use the stable default (Tanh)
    LSTM(64, return_sequences=True, input_shape=(sequence_length, X.shape[2])), 
    Dropout(0.3),
    LSTM(128, return_sequences=True), 
    Dropout(0.3),
    LSTM(64), # Note: Tanh is fine for the final LSTM's main activation
    Dense(64, activation='relu'), # ReLU is fine here in the Dense layers
    Dense(32, activation='relu'),
    Dense(actions.shape[0], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# --- Training ---
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,  # 2000 is overkill; use early stopping
    batch_size=16,
    callbacks=[tb_callback, early_stop, reduce_lr],
    verbose=1
)

# --- Evaluation ---
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Model accuracy: {acc:.4f}")

# --- Save model ---
model.save_weights('model_weights.weights.h5')  # CORRECTED: Must end in .weights.h5
model.save('model_fixed.keras')
print("Model saved successfully.")


✅ Loaded 120 sequences successfully.
Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 10s 187ms/step - categorical_accuracy: 0.4474 - loss: 0.9892 - val_categorical_accuracy: 0.3333 - val_loss: 0.7978 - learning_rate: 0.0010
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - categorical_accuracy: 0.7456 - loss: 0.6657 - val_categorical_accuracy: 0.8333 - val_loss: 0.7915 - learning_rate: 0.0010
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - categorical_accuracy: 0.8684 - loss: 0.5012 - val_categorical_accuracy: 0.8333 - val_loss: 0.7540 - learning_rate: 0.0010
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - categorical_accuracy: 0.9035 - loss: 0.3207 - val_categorical_accuracy: 0.6667 - val_loss: 1.2031 - learning_rate: 0.0010
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 0.8333 - loss: 0.6562 - val_categorical_accuracy: 0.6667 - val_loss: 1.3686 - learning_rate: 0.0010
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - categorical_accuracy: 0.5702 - loss: 1.2724 -